## Raspagem dos resultados das provas da Maratona do Rio 2019 
Meia Maratona, Maratona, Desafio Cidade Maravilhosa, Prova de 5 e 10km e Maratoninha Gloob

In [6]:
import os
import mechanize
from bs4 import BeautifulSoup
import urllib
import pandas as pd
from time import time
from time import sleep
from random import randint

In [14]:
url_base = 'https://globalcronometragem.com.br/evento/'
DATASET_FOLDER  = 'dataset'

eventos = ["meiadorio2019", "maratonadorio2019", "desafiocidademaravilhosa2019"]

br = mechanize.Browser()
br.addheaders = [('User-agent', 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8) AppleWebKit/535.7 (KHTML, like Gecko) Chrome/16.0.912.63 Safari/535.7')]

### Funções Utilitárias

In [8]:
# Recupera a listagem de categorias da prova do evento
def get_categorias(evento):
    
    #Controle de quantidade de requisição
    inicio = time()
    sleep(randint(1,10))
    espera = time() - inicio

    print('--> Request Categoria ' + evento + ' ' + str(espera))
    
    br.open(url_base + evento)
    html = br.response().read()
    
    cat_soup = BeautifulSoup(html, "html.parser")
    select_categorias = cat_soup.find(class_ = "form-control campocategoria")
    
    if select_categorias is None:
        return None
    
    options_categorias = select_categorias.find_all("option")
    
    if options_categorias is None:
        return None
    
    options_categorias = options_categorias[1:]
    
    categorias = []
    
    for o in options_categorias:
        categorias.append(o['value'])
    
    return categorias

In [9]:
#Recupera a URL para o resultado individual de cada atleta do evento por categoria
def get_url_resultados_individuais(lista_categorias, evento, debug = False):
    
    url_resultados_individuais = []

    for categoria in lista_categorias:

        #Controle de quantidade de requisição
        inicio = time()
        sleep(randint(1,10))
        espera = time() - inicio

        print('--> Request ' + categoria + ' ' + str(espera))
        
        params = {'npeito': '', 'campopesquisa': '', 'categoria': categoria}

        data = urllib.parse.urlencode(params)

        br.open(url_base + evento, data = data)
        resp = br.response().read()
        soup = BeautifulSoup(resp, "html.parser")

        div_resultados = soup.find_all(class_ = "campos")

        if div_resultados is None:
            continue

        for resultado in div_resultados:
            a = resultado.find("a")
            
            if a is None:
                continue;
                
            url_individual = a['href']
            
            url_resultados_individuais.append({
                "url": url_individual,
                "categoria": categoria
            })
    
        if debug:
            break
    
    return url_resultados_individuais

In [10]:
#Recupera o resultado individual do atleta
def get_resultado_individual_detalhado(url):

    br.open(url)
    html = br.response().read()

    soup = BeautifulSoup(html, "html.parser")

    div_externa = soup.find_all(class_ = "cimaevento span12")
    
    if div_externa is None:
        return
    
    if len(div_externa) < 2:
        return
    
    div = div_externa[1]

    temp = div.text.split('\n')

    temp = list(filter(lambda x: x != '', temp))[1:-1]

    temp_dic = dict(zip(temp[::2], temp[1::2]))
    
    return temp_dic

In [16]:
#Faz a raspagem de dados do evento e retorna em um dataframe Pandas
def scrape_evento(evento, debug = False):
    
    categorias = get_categorias(evento)

    resultados_categoria = get_url_resultados_individuais(categorias, evento, debug=debug)

    data = []
    count = 0
    
    for linha in resultados_categoria:
        
        if debug == True and count > 5:
            break
        
        #Controle de quantidade de requisição
        inicio = time()
        sleep(randint(1,30))
        espera = time() - inicio

        print('Request ' + str(count) + ' scrape ' + evento + ' ' + str(espera))
        
        count += 1
        
        resultado_individual = get_resultado_individual_detalhado(linha['url'])
        resultado_individual['Categoria'] = linha['categoria']
        data.append(list(resultado_individual.values()))


    columns = ['Nome', 'N. Peito', 'Equipe', 'Modalidade', 'Sexo', 'Colocação Geral', 'Colocação faixa etária','Tempo bruto', 'Tempo líquido', 'Categoria']

    df = pd.DataFrame(data=data, columns=columns)
    
    return df

### Scraping

In [ ]:
for evento in eventos:
    df = scrape_evento(evento, debug=False)
    df.to_csv(DATASET_FOLDER + "/" + evento + ".csv")

--> Request Categoria meiadorio2019 4.0006303787231445
--> Request Masculino de 30 ate 34 anos 4.001173734664917
--> Request Masculino de 25 ate 29 anos 1.001119613647461
--> Request Masculino de 35 ate 39 anos 7.004290819168091
--> Request Masculino de 20 ate 24 anos 1.0011136531829834
--> Request Masculino de 45 ate 49 anos 5.005112409591675
--> Request Masculino de 40 ate 44 anos 9.000494003295898
--> Request Masculino de 50 ate 54 anos 3.0031278133392334
--> Request Masculino de 0 ate 17 anos (Idade invalida) 7.0066375732421875
--> Request Masculino de 55 ate 59 anos 2.0021190643310547
--> Request Masculino de 60 ate 64 anos 3.0031304359436035
--> Request Masculino de 65 ate 69 anos 5.003076791763306
--> Request Masculino de 75 ate 79 anos 1.001115083694458
--> Request Masculino de 18 ate 19 anos 9.009131908416748
--> Request Masculino de 70 ate 74 anos 9.009122848510742
--> Request Masculino de 80 anos em diante 6.006115913391113
--> Request te 4.0041210651397705
--> Request Femin

Request 128 scrape meiadorio2019 16.002373695373535
Request 129 scrape meiadorio2019 1.0011253356933594
Request 130 scrape meiadorio2019 16.012707233428955
Request 131 scrape meiadorio2019 12.012125968933105
Request 132 scrape meiadorio2019 30.030126571655273
Request 133 scrape meiadorio2019 21.019290685653687
Request 134 scrape meiadorio2019 3.0030946731567383
Request 135 scrape meiadorio2019 24.01300573348999
Request 136 scrape meiadorio2019 11.008503437042236
Request 137 scrape meiadorio2019 30.005233764648438
Request 138 scrape meiadorio2019 18.017043352127075
Request 139 scrape meiadorio2019 27.00509238243103
Request 140 scrape meiadorio2019 17.013976573944092
Request 141 scrape meiadorio2019 19.019116640090942
Request 142 scrape meiadorio2019 20.006960153579712
Request 143 scrape meiadorio2019 25.004998922348022
Request 144 scrape meiadorio2019 29.004742860794067
Request 145 scrape meiadorio2019 8.008126020431519
Request 146 scrape meiadorio2019 7.003347396850586
Request 147 scra

Request 287 scrape meiadorio2019 15.015120029449463
Request 288 scrape meiadorio2019 23.002285957336426
Request 289 scrape meiadorio2019 10.002037763595581
Request 290 scrape meiadorio2019 27.02712631225586
Request 291 scrape meiadorio2019 1.0011239051818848
Request 292 scrape meiadorio2019 17.012091159820557
Request 293 scrape meiadorio2019 3.003129005432129
Request 294 scrape meiadorio2019 29.029072523117065
Request 295 scrape meiadorio2019 19.01912498474121
Request 296 scrape meiadorio2019 2.002119541168213
Request 297 scrape meiadorio2019 3.0004703998565674
Request 298 scrape meiadorio2019 15.00860047340393
Request 299 scrape meiadorio2019 27.0035240650177
Request 300 scrape meiadorio2019 22.022128105163574
Request 301 scrape meiadorio2019 9.0059814453125
Request 302 scrape meiadorio2019 20.020121335983276
Request 303 scrape meiadorio2019 29.005491256713867
Request 304 scrape meiadorio2019 17.015050172805786
Request 305 scrape meiadorio2019 17.00546431541443
Request 306 scrape meia

Request 447 scrape meiadorio2019 1.00069260597229
Request 448 scrape meiadorio2019 26.0261173248291
Request 449 scrape meiadorio2019 4.004124402999878
Request 450 scrape meiadorio2019 4.00412654876709
Request 451 scrape meiadorio2019 5.005110263824463
Request 452 scrape meiadorio2019 8.004417896270752
Request 453 scrape meiadorio2019 21.003682374954224
Request 454 scrape meiadorio2019 5.0003502368927
Request 455 scrape meiadorio2019 9.002577781677246
Request 456 scrape meiadorio2019 27.026885271072388
Request 457 scrape meiadorio2019 22.012696266174316
Request 458 scrape meiadorio2019 24.00588369369507
Request 459 scrape meiadorio2019 30.027233362197876
Request 460 scrape meiadorio2019 7.0071141719818115
Request 461 scrape meiadorio2019 12.012122392654419
Request 462 scrape meiadorio2019 30.012247800827026
Request 463 scrape meiadorio2019 22.014230728149414
Request 464 scrape meiadorio2019 19.012452602386475
Request 465 scrape meiadorio2019 30.028355360031128
Request 466 scrape meiador

Request 607 scrape meiadorio2019 1.0011317729949951
Request 608 scrape meiadorio2019 18.015501260757446
Request 609 scrape meiadorio2019 19.019063234329224
Request 610 scrape meiadorio2019 10.010080337524414
Request 611 scrape meiadorio2019 7.0014636516571045
Request 612 scrape meiadorio2019 17.016742944717407
Request 613 scrape meiadorio2019 2.0008888244628906
Request 614 scrape meiadorio2019 26.015308141708374
Request 615 scrape meiadorio2019 17.014352560043335
Request 616 scrape meiadorio2019 20.011191606521606
Request 617 scrape meiadorio2019 1.001033067703247
Request 618 scrape meiadorio2019 14.003731966018677
Request 619 scrape meiadorio2019 4.000743389129639
Request 620 scrape meiadorio2019 9.006976127624512
Request 621 scrape meiadorio2019 15.011933088302612
Request 622 scrape meiadorio2019 22.011025428771973
Request 623 scrape meiadorio2019 12.008689641952515
Request 624 scrape meiadorio2019 28.00155520439148
Request 625 scrape meiadorio2019 19.007755756378174
Request 626 scra

Request 767 scrape meiadorio2019 5.00512433052063
Request 768 scrape meiadorio2019 25.02512288093567
Request 769 scrape meiadorio2019 29.011666297912598
Request 770 scrape meiadorio2019 4.00177264213562
Request 771 scrape meiadorio2019 3.002187728881836
Request 772 scrape meiadorio2019 18.01755380630493
Request 773 scrape meiadorio2019 28.026261568069458
Request 774 scrape meiadorio2019 23.007492065429688
Request 775 scrape meiadorio2019 1.001081943511963
Request 776 scrape meiadorio2019 2.0020997524261475
Request 777 scrape meiadorio2019 9.002309083938599
Request 778 scrape meiadorio2019 30.02604031562805
Request 779 scrape meiadorio2019 7.000697135925293
Request 780 scrape meiadorio2019 10.010132551193237
Request 781 scrape meiadorio2019 18.01811933517456
Request 782 scrape meiadorio2019 10.003000736236572
Request 783 scrape meiadorio2019 28.027662754058838
Request 784 scrape meiadorio2019 26.01443099975586
Request 785 scrape meiadorio2019 1.0011191368103027
Request 786 scrape meiado

In [7]:
df_maratona = pd.DataFrame.from_csv(DATASET_FOLDER + "/" + eventos[1] + ".csv")

In [8]:
df_meia = pd.DataFrame.from_csv(DATASET_FOLDER + "/" + eventos[0] + ".csv")

In [17]:
df = df_meia.append(df_maratona)

In [20]:
df.reset_index(drop=True)

,Nome,N. Peito,Equipe,Modalidade,Sexo,Colocação Geral,Colocação faixa etária,Tempo bruto,Tempo líquido,Categoria
0,STANLEY KIPCHIRCHIR KOECH,15011,Sem equipe,21,M,1,0,01:04:38,01:04:34,Masculino de 30 ate 34 anos
1,GILMAR SILVESTRE LOPES,15008,Sem equipe,21,M,2,0,01:04:42,01:04:39,Masculino de 30 ate 34 anos
2,ANDRÃ LUIZ SILVA ANTONIO,15001,Sem equipe,21,M,5,0,01:10:02,01:09:59,Masculino de 30 ate 34 anos
3,JACOB KEMBOI KIPROTICH,15015,Sem equipe,21,M,8,0,01:11:52,01:11:48,Masculino de 30 ate 34 anos
4,GIOVANI GERONASSO,24207,V8 ASSESSORIA ESPORTIVA,21,M,29,1,01:25:29,01:19:58,Masculino de 30 ate 34 anos
5,JOSIAS NEPOMUCENO RIBEIRO,25848,CORRIDA PARATODOS,21,M,31,2,01:22:15,01:20:11,Masculino de 30 ate 34 anos
6,GEOVANI DOS SANTOS,36,APPAI / 361,42,M,1,0,02:18:48,02:18:47,Masculino de 0 ate 19 anos (Idade invalida)
7,ANTONIO WILSON SOUSA LIMA,2,TJOUTLETRUNNING,42,M,2,0,02:20:05,02:20:05,Masculino de 0 ate 19 anos (Idade invalida)
8,JONATAS ROGER FREITAS,18,Sem equipe,42,M,47,0,02:48:56,02:48:54,Masculino de 0 ate 19 anos (Idade invalida)
9,JOSÃ GILDO DE ARAÃJO SILVA,1121,Sem equipe,42,M,84,0,02:58:26,02:58:26,Masculino de 0 ate 19 anos (Idade invalida)
